# MAGIC+LST-1 data quality 

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import math
import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator, NullFormatter
import re
from ctapipe.io import read_table
from astropy.table import Table

from astropy.coordinates import SkyCoord, AltAz, angular_separation
import astropy.units as u

from gammapy.data import DataStore, EventList
from astropy.io import fits
from matplotlib.ticker import FuncFormatter
from matplotlib.colors import Normalize

from scipy.optimize import curve_fit

In [ ]:
# ===========
#   INPUT
# ===========

# source name and coordinates (ra [deg], dec [deg])

# ----------------------------
obj_name = "Crab"
source_ra = 83.6324
source_dec = 22.0174
# ----------------------------

target_position = SkyCoord.from_name(obj_name, frame="icrs")
target_position

# LST-1 section

In [ ]:
# ===========
#   INPUT
# ===========

# LST-1 dl1 files
# LST-1 datacheck

lst_file_paths = [
    "/fefs/aswg/data/real/DL1/20221218/v0.10/tailcut84/dl1_LST-1.Run11466.h5",
    "/fefs/aswg/data/real/DL1/20221218/v0.10/tailcut84/dl1_LST-1.Run11464.h5",
    "/fefs/aswg/data/real/DL1/20221218/v0.10/tailcut84/dl1_LST-1.Run11469.h5",
    "/fefs/aswg/data/real/DL1/20221217/v0.10/tailcut84/dl1_LST-1.Run11430.h5",
    "/fefs/aswg/data/real/DL1/20221217/v0.10/tailcut84/dl1_LST-1.Run11430.h5",
    "/fefs/aswg/data/real/DL1/20221217/v0.10/tailcut84/dl1_LST-1.Run11431.h5",
]

files_datacheck = glob.glob(
    "/fefs/aswg/data/real/OSA/DL1DataCheck_LongTerm/night_wise/all/DL1_datacheck_2022*.h5"
)

In [ ]:
lst_run_ids = []

for file in lst_file_paths:
    run_id_str = file.split("Run")[-1].split(".")[0]
    run_id = int(run_id_str.lstrip("0"))
    lst_run_ids.append(run_id)

print(lst_run_ids)

## Pointing stability

In [ ]:
max_std_dec = 0.01

all_data = []

for file_d in files_datacheck:
    try:
        df = pd.read_hdf(file_d, key="cosmics_intensity_spectrum/table")
        all_data.append(df)
    except Exception as e:
        print(f" Something went wrong: {file_d}: {e}")

df_all = pd.concat(all_data, ignore_index=True)

if "runnumber" not in df_all.columns or "dec_tel" not in df_all.columns:
    raise ValueError("No columns: 'runnumber' and 'dec_tel'")

grouped = df_all.groupby("runnumber")["dec_tel"]
stats = grouped.agg(["mean", "std"]).reset_index()
stats.rename(columns={"mean": "mean_dec", "std": "std_dec"}, inplace=True)

max_std_dec = 0.01
stats["stable"] = stats["std_dec"] <= max_std_dec

for _, row in stats.iterrows():
    run = int(row["runnumber"])
    for r in lst_run_ids:
        if r == run:
            mean_dec = row["mean_dec"]
            std_dec = row["std_dec"]
            status = "STABLE" if row["stable"] else "UNSTABLE"
            print(
                f"Run {run}: mean_dec = {mean_dec:.5f} deg, std_dec = {std_dec:.5f} deg, {status}"
            )

## Event rate LST-1

In [ ]:
min_time = 300  # [s]
# intensity cut (default: >50 phe)
phe = 50

n_files = len(lst_file_paths)
n_cols = 2
n_rows = int(np.ceil(n_files / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4 * n_rows), sharey=False)
plt.subplots_adjust(wspace=0.3)

if n_rows == 1:
    axes = np.array([axes])

for idx, file in enumerate(lst_file_paths):
    row = idx // n_cols
    col = idx % n_cols

    match = re.search(r"\.Run0*(\d+)\.h5$", file)
    run_number = int(match.group(1))

    hillas_params = read_table(file, "/dl1/event/telescope/parameters/LST_LSTCam")

    filtered = hillas_params[hillas_params["intensity"] > phe]
    times = filtered["trigger_time"] - filtered["trigger_time"].min()

    bin_width = 1  # [s]
    bins = np.arange(0, times.max() + bin_width, bin_width)
    counts, bin_edges = np.histogram(times, bins=bins)
    event_rate = counts / bin_width

    std_dev = np.std(event_rate)
    mean_rate = np.mean(event_rate)
    relative_fluctuation = (std_dev / mean_rate) * 100

    ax = axes[row, col]
    ax.step(bin_edges[:-1], event_rate, where="mid")
    ax.axvline(x=min_time, color="green", linestyle="--")
    # ax.set_ylim(0, 6000)
    ax.set_xlabel("Time since start of run [s]")
    ax.set_ylabel("Event rate [Hz]")
    ax.set_title(
        f"Event rate vs time for {run_number} (intensity > {phe} phe)\n Mean: {mean_rate:.1f} Hz, Fluctuation: {relative_fluctuation:.1f}%"
    )
    ax.grid()


for idx in range(n_files, n_rows * n_cols):
    fig.delaxes(axes.flatten()[idx])

plt.tight_layout()
plt.show()

## Intensity distribution

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

# for file_path in lst_file_paths[:10]: #if you want a range
for file_path in lst_file_paths:
    match = re.search(r"\.Run0*(\d+)\.h5$", file_path)
    if not match:
        print(f"Something went wrong with: {file_path}")
        continue
    run_number = int(match.group(1))

    df = pd.read_hdf(file_path, key="/dl1/event/telescope/parameters/LST_LSTCam")

    df_selected = df[["intensity", "event_type"]].dropna(subset=["intensity"])
    df_filtered = df_selected[df_selected["event_type"] == 32]

    if df_filtered.empty:
        print(f"Empty dataset in run {run_number}.")
        continue

    min_intensity = df_filtered["intensity"].min()
    max_intensity = df_filtered["intensity"].max()
    bins = np.logspace(np.log10(min_intensity), np.log10(max_intensity), 50)

    ax.hist(
        df_filtered["intensity"], bins=bins, histtype="step", label=f"Run {run_number}"
    )

ax.set_xscale("log")
ax.set_yscale("log")

minor_subs = np.arange(2, 10) * 0.1
ax.xaxis.set_major_locator(LogLocator(base=10.0, subs=[1.0]))
ax.yaxis.set_major_locator(LogLocator(base=10.0, subs=[1.0]))
ax.xaxis.set_minor_locator(LogLocator(base=10.0, subs=minor_subs))
ax.yaxis.set_minor_locator(LogLocator(base=10.0, subs=minor_subs))
ax.xaxis.set_minor_formatter(NullFormatter())
ax.yaxis.set_minor_formatter(NullFormatter())

ax.grid(True, which="major", linestyle="--", linewidth=0.5)
ax.grid(False, which="minor")

ax.set_xlabel("Intensity [p.e.]")
ax.set_ylabel("Counts")
ax.set_title("Intensity distribution")
ax.legend()

ax.set_xlim(left=1e0, right=1e5)

plt.tight_layout()
plt.show()

## Cosmic rate at 422 p.e

It is best to load as much data as possible (files_datacheck) to get a broader view.

In [ ]:
dfs = []
for file_d in files_datacheck:
    try:
        df = pd.read_hdf(file_d, key="cosmics_intensity_spectrum/table")
        dfs.append(df)
    except Exception as e:
        print(f"Problem with file {file_d}: {e}")

df_all = pd.concat(dfs, ignore_index=True)

In [ ]:
agg = df_all.groupby("runnumber", as_index=False).agg(
    mean_rate=("cosmics_rate_at_422_pe", "mean"),
    n_subruns=("cosmics_rate_at_422_pe", "count"),
)

lst_run_ids_int = list(map(int, lst_run_ids))
highlighted = agg[agg["runnumber"].astype(int).isin(lst_run_ids_int)]
others = agg[~agg["runnumber"].astype(int).isin(lst_run_ids_int)]

plt.figure(figsize=(12, 5))
plt.scatter(
    others["runnumber"], others["mean_rate"], color="blue", label="Other runs", s=20
)
plt.scatter(
    highlighted["runnumber"],
    highlighted["mean_rate"],
    color="orange",
    label="Selected runs",
    s=35,
)
plt.ylim(0, 3)

mean_val = agg["mean_rate"].mean()
plt.axhline(mean_val, color="red", linestyle="--", label=f"Mean: {mean_val:.2f} Hz")

plt.xlabel("Run number")
plt.ylabel("Cosmic rate @422 pe [Hz] (run-averaged)")
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# thresholds
lower = 0.8
upper = 1.7

# outliers (only selected runs)
outliers = highlighted[
    (highlighted["mean_rate"] < lower) | (highlighted["mean_rate"] > upper)
]

if outliers.empty:
    print("No selected runs outside thresholds.")
else:
    print("Selected runs outside thresholds:")
    print(outliers[["runnumber", "mean_rate"]].to_string(index=False))

# MAGIC + LST-1 section

In [ ]:
# ===========
#   INPUT
# ===========

# dl2_path = " your/data/path "
# ----------------------------
dl2_path = "/fefs/aswg/LST1MAGIC/data/v0.5.6/*/DL2/2022121[7-8]/"
# ----------------------------

pattern_dl2 = os.path.join(dl2_path, "dl2_MAGIC_LST-1.Run*.h5")
dl2_files = glob.glob(pattern_dl2)

print(f"{len(dl2_files)} files found.")

In [ ]:
# ===========
#   INPUT
# ===========

# dl3_path = "your/data/path"
# ----------------------------
dl3_path = "/fefs/aswg/LST1MAGIC/data/v0.5.7/*/DL3/g_dyn_0.9_th_glo_0.2/2022121[7-8]/"
# ----------------------------

pattern_dl3 = os.path.join(dl3_path, "dl3_MAGIC_LST-1.Run*.fits.gz")
dl3_files = glob.glob(pattern_dl3)
print(f"{len(dl3_files)} files found.")

## Zenith angle (no cuts yet)

In [ ]:
run_ids = []
zenith_min = []
zenith_max = []

for filepath in dl3_files:
    match = re.search(r"Run(\d+)", os.path.basename(filepath))
    if not match:
        continue
    run_id = int(match.group(1))

    try:
        with fits.open(filepath) as hdul:
            data = hdul["EVENTS"].data
            alt = data["ALT"]
            zenith = 90.0 - alt
            zenith_min.append(np.min(zenith))
            zenith_max.append(np.max(zenith))
            run_ids.append(run_id)
    except Exception as e:
        print(f"Error in the file {filepath}: {e}")
        continue

run_ids = np.array(run_ids)
zenith_min = np.array(zenith_min)
zenith_max = np.array(zenith_max)

zenith_range = zenith_max - zenith_min

x_pos = np.arange(len(run_ids))

fig, ax = plt.subplots(figsize=(10, 6))

ax.vlines(x_pos, zenith_min, zenith_max, color="black", linewidth=1)

ax.plot(x_pos, zenith_min, "bo")
ax.plot(x_pos, zenith_max, "bo")

zenith_limit = 60
ax.axhline(zenith_limit, color="red", linestyle="-", linewidth=2)
ax.fill_between(
    x_pos, zenith_limit, zenith_limit + (90 - zenith_limit), color="red", alpha=0.2
)

ax.set_xticks(x_pos)
ax.set_xticklabels(run_ids, rotation=90, ha="right")
ax.set_xlabel("Run ID")
ax.set_ylabel("Zenith angle [deg]")
ax.set_title("Zenith angle range per run")
ax.grid(True, linestyle="--", alpha=0.5)
ax.legend()
plt.tight_layout()
plt.show()

## NSB level

In [ ]:
LST_database = "/fefs/aswg/workspace/elisa.visentin/auto_MCP_PR/observations_LST.h5"
LST_database_key = "joint_obs"
df_LST = pd.read_hdf(LST_database, key=LST_database_key)

nsb_values = []
runs_low_nsb = []
runs_high_nsb = []

df_LST["LST1_run"] = df_LST["LST1_run"].astype(int)

for run_id in run_ids:

    match = df_LST[df_LST["LST1_run"] == run_id]

    if not match.empty:
        nsb = match["nsb"].iloc[0]
    else:
        nsb = None
    nsb_values.append(nsb)

    if nsb <= 1.75:
        runs_low_nsb.append(run_id)
    else:
        runs_high_nsb.append(run_id)

run_ids_str = [str(rid) for rid in run_ids]


plt.figure(figsize=(10, 5))
plt.scatter(run_ids_str, nsb_values)
plt.axhline(y=1.75, color="green", linestyle="--")

plt.xlabel("Run ID")
plt.ylabel("NSB")
plt.title("NSB per Run ID")
plt.grid(True, axis="y")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
print("Runs with nsb <= 1.75")
print(runs_low_nsb)
print()
print("Runs with nsb > 1.75")
print(runs_high_nsb)

## Countmap

In [ ]:
# ===========
#   INPUT
# ===========

# ra/dec range on your map
# number of bins

# ----------------------------
ra_min, ra_max = 83, 84.2
dec_min, dec_max = 21.6, 22.4
n_ra_bins = 40
n_dec_bins = 40
# ----------------------------

In [ ]:
all_ra = []
all_dec = []

for filename in dl3_files:
    with fits.open(filename) as hdul:
        data = hdul["EVENTS"].data
        ra = data["RA"]
        dec = data["DEC"]
        mask = (ra >= ra_min) & (ra <= ra_max) & (dec >= dec_min) & (dec <= dec_max)
        all_ra.append(ra[mask])
        all_dec.append(dec[mask])

    if len(all_ra) == 0 or len(all_dec) == 0:
        print("No counts in the RA/DEC range.")
    else:
        ra_sel = np.concatenate(all_ra)
        dec_sel = np.concatenate(all_dec)

        print(f"Total counts: {len(ra_sel)}.")

        H, xedges, yedges = np.histogram2d(
            ra_sel,
            dec_sel,
            bins=(n_ra_bins, n_dec_bins),
            range=[[ra_min, ra_max], [dec_min, dec_max]],
        )
        X, Y = np.meshgrid(xedges, yedges)

        fig, ax = plt.subplots(figsize=(8, 6))
        mesh = ax.pcolormesh(X, Y, H.T, cmap="YlOrBr_r")

        def deg_to_hms(x, pos):
            h = int(x / 15)
            m = int((x / 15 - h) * 60)
            return f"{h}ʰ{m:02d}ᵐ"

        ax.xaxis.set_major_formatter(FuncFormatter(deg_to_hms))
        ax.set_xlim(ra_max, ra_min)

        ax.plot(
            source_ra, source_dec, "o", color="limegreen", markersize=6, label="Source"
        )

        plt.colorbar(mesh, ax=ax, label="Counts")
        ax.set_xlabel("RA")
        ax.set_ylabel("Dec")
        ax.set_title(f"Countmap \n {filename}")
        ax.legend()
        plt.tight_layout()
        plt.show()

## Event rate from dl2 files

In [ ]:
all_dfs = []
run_ids_list = []

# for file in dl2_files[:]: # if you want a range
for file in dl2_files:
    filename = os.path.basename(file)
    run_id_str = filename.split("Run")[-1].split(".")[0]
    run_id = int(run_id_str.lstrip("0"))

    events_table = read_table(file, "/events/parameters")

    df = pd.DataFrame(
        {
            "event_id": events_table["event_id_lst"],
            "timestamp": events_table["timestamp"],
            "zenith": 90 - events_table["alt"],
            "run_id": run_id,
        }
    )

    df_unique = df[~df["event_id"].duplicated(keep="first")].reset_index(drop=True)
    all_dfs.append((df_unique, run_id))

n_files = len(all_dfs)
print(n_files)
n_cols = 2
n_rows = math.ceil(n_files / n_cols)


fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4 * n_rows), sharex=False)
axes = axes.flatten()

for idx, (df, run_id) in enumerate(all_dfs):

    event_rates = []

    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    min_time = df["timestamp"].min()
    df["time_since_start"] = (df["timestamp"] - min_time).dt.total_seconds()
    times = df["time_since_start"].values

    if times.size == 0:
        print(f"Run {run_id} has 0 events.")
        continue

    run_ids_list.append(run_id)
    bin_width = 1  # [s]
    bins = np.arange(0, times.max() + bin_width, bin_width)
    counts, bin_edges = np.histogram(times, bins=bins)
    event_rate = counts / bin_width
    event_rates.append(event_rate)

    ax = axes[idx]
    ax.step(bin_edges[:-1], event_rate, where="mid")
    ax.axvline(x=300, color="green", linestyle="--")  # 300s = 5 min
    ax.set_xlabel("Time since start of run [s]")
    ax.set_ylabel("Event rate [Hz]")
    ax.set_ylim(0, 300)

    ax.set_title(f"Event rate vs time for Run {run_id}", fontsize=10)
    ax.grid()

for idx in range(n_files, n_rows * n_cols):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.show()

## Event rate vs zenith angle (+zenith cut)

In [ ]:
# ===========
#   INPUT
# ===========

# zenith cut [deg]

# ------------------
zenith_limit = 60
# ------------------

### MAGIC_obs_periods: 
        ST0316A: [["2020_10_24", "2021_09_29"]]
        ST0317A: [["2021_12_30", "2022_06_09"]]       
        ST0318A: [["2022_06_10", "2022_08_31"], ["2022_12_15", "2023_01_06"]]
        M20318A: [["2022_09_04", "2022_12_14"]]
        ST0319A: [["2023_01_07", "2023_03_09"]]
        ST0320A: [["2023_03_10", "2024_05_18"]]
        ST0321A: [["2024_05_19", "2026_01_01"]]
        # ST0321 ongoing -> 'service' end date

In [ ]:
# selection run id by MAGIC obs. periods

ST0316A = []
ST0316A_mean_event_rates_list = []
ST0316A_zenith_mean_list = []

ST0317A = []
ST0317A_mean_event_rates_list = []
ST0317A_zenith_mean_list = []

ST0318A = []
ST0318A_mean_event_rates_list = []
ST0318A_zenith_mean_list = []

M20318A = []
M20318A_mean_event_rates_list = []
M20318A_zenith_mean_list = []

ST0319A = []
ST0319A_mean_event_rates_list = []
ST0319A_zenith_mean_list = []

ST0320A = []
ST0320A_mean_event_rates_list = []
ST0320A_zenith_mean_list = []

ST0321A = []
ST0321A_mean_event_rates_list = []
ST0321A_zenith_mean_list = []


for df, run_id in all_dfs:
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    t0 = df["timestamp"].min()
    times = (df["timestamp"] - t0).dt.total_seconds().to_numpy()
    start_run_date = df["timestamp"].iloc[0]

    if times.size == 0:
        continue

    bin_width = 1
    bins = np.arange(0, times.max() + bin_width, bin_width)
    counts, _ = np.histogram(times, bins=bins)
    event_rate_per_bin = counts / bin_width  # [Hz]

    event_rate_nonzero = event_rate_per_bin[event_rate_per_bin > 0]
    if event_rate_nonzero.size == 0:
        continue

    mean_rate = np.mean(event_rate_nonzero)

    zrun = df["zenith"].to_numpy(dtype=float)
    if zrun.size == 0:
        continue
    zenith_mean = np.nanmean(zrun)

    if zenith_mean <= zenith_limit:

        if (start_run_date >= pd.to_datetime("2020-10-24")) and (
            start_run_date < pd.to_datetime("2021-09-29")
        ):
            ST0316A.append(run_id)
            ST0316A_mean_event_rates_list.append(float(mean_rate))
            ST0316A_zenith_mean_list.append(float(zenith_mean))
        elif (start_run_date >= pd.to_datetime("2021-12-30")) and (
            start_run_date < pd.to_datetime("2022-06-09")
        ):
            ST0317A.append(run_id)
            ST0317A_mean_event_rates_list.append(float(mean_rate))
            ST0317A_zenith_mean_list.append(float(zenith_mean))
        elif (start_run_date >= pd.to_datetime("2022-06-10")) and (
            start_run_date < pd.to_datetime("2022-08-31")
        ):
            ST0318A.append(run_id)
            ST0318A_mean_event_rates_list.append(float(mean_rate))
            ST0318A_zenith_mean_list.append(float(zenith_mean))
        elif (start_run_date >= pd.to_datetime("2022-12-15")) and (
            start_run_date < pd.to_datetime("2023-01-06")
        ):
            ST0318A.append(run_id)
            ST0318A_mean_event_rates_list.append(float(mean_rate))
            ST0318A_zenith_mean_list.append(float(zenith_mean))
        elif (start_run_date >= pd.to_datetime("2022-09-04")) and (
            start_run_date < pd.to_datetime("2022-12-14")
        ):
            M20318A.append(run_id)
            M20318A_mean_event_rates_list.append(float(mean_rate))
            M20318A_zenith_mean_list.append(float(zenith_mean))
        elif (start_run_date >= pd.to_datetime("2023-01-07")) and (
            start_run_date < pd.to_datetime("2023-03-09")
        ):
            ST0319A.append(run_id)
            ST0319A_mean_event_rates_list.append(float(mean_rate))
            ST0319A_zenith_mean_list.append(float(zenith_mean))
        elif (start_run_date >= pd.to_datetime("2023-03-10")) and (
            start_run_date < pd.to_datetime("2024-05-18")
        ):
            ST0320A.append(run_id)
            ST0320A_mean_event_rates_list.append(float(mean_rate))
            ST0320A_zenith_mean_list.append(float(zenith_mean))
        elif (start_run_date >= pd.to_datetime("2024-05-19")) and (
            start_run_date < pd.to_datetime("2026-01-01")
        ):
            ST0321A.append(run_id)
            ST0321A_mean_event_rates_list.append(float(mean_rate))
            ST0321A_zenith_mean_list.append(float(zenith_mean))

In [ ]:
print(ST0318A)

In [ ]:
common = set(ST0318A) & set(runs_high_nsb)
print(sorted(common))

In [ ]:
def sort_runs_by_zenith(run_ids, zeniths, rates):
    data_sorted = sorted(zip(zeniths, run_ids, rates), key=lambda x: x[0])
    return data_sorted


for label, run_ids, zeniths, rates in [
    ("ST0316A", ST0316A, ST0316A_zenith_mean_list, ST0316A_mean_event_rates_list),
    ("ST0317A", ST0317A, ST0317A_zenith_mean_list, ST0317A_mean_event_rates_list),
    ("ST0318A", ST0318A, ST0318A_zenith_mean_list, ST0318A_mean_event_rates_list),
    ("M20318A", M20318A, M20318A_zenith_mean_list, M20318A_mean_event_rates_list),
    ("ST0319A", ST0319A, ST0319A_zenith_mean_list, ST0319A_mean_event_rates_list),
    ("ST0320A", ST0320A, ST0320A_zenith_mean_list, ST0320A_mean_event_rates_list),
    ("ST0321A", ST0321A, ST0321A_zenith_mean_list, ST0321A_mean_event_rates_list),
]:
    print(f"\n{label}:")
    for zen, rid, rate in sort_runs_by_zenith(run_ids, zeniths, rates):
        print(f"Run {rid}, Zenith = {zen:.2f}, Mean rate = {rate:.2f} Hz")

In [ ]:
print("green dots: nsb <= 1.75")
print("blue dots: nsb > 1.75")

# model: A * cos^n(zenith)
def cos_model(z, A, n):
    return A * np.cos(np.radians(z)) ** n


groups = {
    "ST0316A": (ST0316A_zenith_mean_list, ST0316A_mean_event_rates_list, ST0316A),
    "ST0317A": (ST0317A_zenith_mean_list, ST0317A_mean_event_rates_list, ST0317A),
    "ST0318A": (ST0318A_zenith_mean_list, ST0318A_mean_event_rates_list, ST0318A),
    "M20318A": (M20318A_zenith_mean_list, M20318A_mean_event_rates_list, M20318A),
    "ST0320A": (ST0320A_zenith_mean_list, ST0320A_mean_event_rates_list, ST0320A),
    "ST0321A": (ST0321A_zenith_mean_list, ST0321A_mean_event_rates_list, ST0321A),
}

manual_params = {
    "ST0316A": dict(A=124.72, n=0.19, lower=0.3, upper=0.50),
    "ST0317A": dict(A=108.94, n=0.21, lower=0.3, upper=0.5),
    "ST0318A": dict(A=92.2, n=0.53, lower=0.3, upper=0.5),
    "M20318A": dict(A=160.13, n=0.45, lower=0.3, upper=0.5),
    "ST0320A": dict(A=92.35, n=0.21, lower=0.4, upper=0.60),
    "ST0321A": dict(A=124.22, n=0.5, lower=0.3, upper=0.50),
}

fig, axes = plt.subplots(2, 3, figsize=(18, 10), sharey=True)
axes = axes.flatten()

for idx, (label, (zeniths, rates, run_ids)) in enumerate(groups.items()):
    ax = axes[idx]

    if len(zeniths) == 0:
        ax.set_title(f"{label} (no data)")
        continue

    z = np.array(zeniths, dtype=float)
    r = np.array(rates, dtype=float)

    for zi, ri, rid in zip(z, r, run_ids):
        if rid in runs_low_nsb:
            color = "lightgreen"
        elif rid in runs_high_nsb:
            color = "blue"
        else:
            color = "gray"
        ax.scatter(zi, ri, color=color, edgecolors="k")

    params = manual_params[label]
    A_manual = params["A"]
    n_manual = params["n"]
    lower_frac = params["lower"]
    upper_frac = params["upper"]

    z_fit = np.linspace(np.min(z), np.max(z), 200)
    r_fit = cos_model(z_fit, A_manual, n_manual)

    r_lower = r_fit * (1 - lower_frac)
    r_upper = r_fit * (1 + upper_frac)

    ax.plot(
        z_fit, r_fit, "r-", lw=2, label=rf"$A \cos^n(z)$, A={A_manual}, n={n_manual}"
    )
    ax.fill_between(
        z_fit,
        r_lower,
        r_upper,
        color="green",
        alpha=0.2,
        label=f"−{int(lower_frac*100)}% / +{int(upper_frac*100)}%",
    )

    ax.set_title(label)
    ax.set_ylim(0, 250)
    ax.set_xlabel("Zenith angle [deg]")
    if idx % 3 == 0:
        ax.set_ylabel("Mean event rate [Hz]")
    ax.grid(True, linestyle="--", alpha=0.5)
    ax.legend(fontsize=8)

for j in range(idx + 1, len(axes)):
    axes[j].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
print(f"Runs out of range:")

for label, (zeniths, rates, run_ids) in groups.items():
    if len(zeniths) == 0:
        continue

    z = np.array(zeniths, dtype=float)
    r = np.array(rates, dtype=float)
    run_ids = np.array(run_ids)

    params = manual_params[label]
    A_manual = params["A"]
    n_manual = params["n"]
    lower_frac = params["lower"]
    upper_frac = params["upper"]

    r_expected = cos_model(z, A_manual, n_manual)
    r_lower = r_expected * (1 - lower_frac)
    r_upper = r_expected * (1 + upper_frac)

    print()
    for rid, zi, ri, rlo, rhi in zip(run_ids, z, r, r_lower, r_upper):
        if ri < rlo or ri > rhi:
            print(f"[{label}] Run {rid}, Zenith={zi:.2f}, " f"Rate={ri:.2f} Hz ")

## some additional graphs

In [ ]:
events = EventList.read(dl3_files[5])

In [ ]:
events.table[:10]

In [ ]:
for fi in dl3_files[:2]:
    print(fi)
    events.peek()